In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\ADMIN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install torch_geometric

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\ADMIN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
%%capture
!kaggle datasets download -d vitvhong/uplink-sum-rate
!unzip uplink-sum-rate.zip -d ./uplink
%cd uplink

In [4]:
import numpy as np
import torch
from torch_geometric.loader import  DataLoader
from torch_geometric.data import HeteroData, Data, Dataset, Batch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
import torch.nn as nn
from torch.nn import Sequential as Seq, Linear as Lin, ReLU, ELU, Sigmoid, BatchNorm1d as BN, ReLU6 as ReLU6
import scipy.io
from torch_geometric.nn import HeteroConv, SAGEConv
import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

In [5]:
train_layouts = 5000
test_layouts = 100

In [ ]:
import scipy.io
import numpy as np

# Danh sách tên các tệp .mat mà bạn muốn tải
file_names = ['cf_train_20_50_sum.mat', 'cf_train_20_70_sum.mat', 'cf_train_20_100_sum.mat']

# Khởi tạo danh sách để lưu trữ dữ liệu từ mỗi tệp
data_list = []

# Lặp qua từng tệp để tải dữ liệu
for file_name in file_names:
    # Tải dữ liệu từ tệp
    data = scipy.io.loadmat(file_name)

    # Lấy mẫu từ các biến
    beta_samples = data['betas'][:train_layouts]
    direct_samples = data['directs'][:train_layouts]
    cross_samples = data['corsses'].transpose(0, 2, 1)[:train_layouts]

    # Lưu trữ dữ liệu vào từ điển
    data_list.append({
        'beta': beta_samples,
        'direct': direct_samples,
        'cross': cross_samples
    })

# Bây giờ, data_list chứa dữ liệu từ tất cả các tệp, bạn có thể truy cập từng tệp như sau:
for i, file_data in enumerate(data_list):
    print(f"File {i + 1}:")
    print("Beta shape:", file_data['beta'].shape)
    print("Direct shape:", file_data['direct'].shape)
    print("Cross shape:", file_data['cross'].shape)
    print()


File 1:
Beta shape: (5000, 30, 10)
Direct shape: (5000, 10)
Cross shape: (5000, 10, 10)

File 2:
Beta shape: (5000, 50, 20)
Direct shape: (5000, 20)
Cross shape: (5000, 20, 20)

File 3:
Beta shape: (5000, 70, 10)
Direct shape: (5000, 10)
Cross shape: (5000, 10, 10)

File 4:
Beta shape: (5000, 100, 20)
Direct shape: (5000, 20)
Cross shape: (5000, 20, 20)



In [7]:
test_data = scipy.io.loadmat('30_10.mat')
beta_test = test_data['betas']
direct_test = test_data['directs']
cross_test = test_data['corsses'].transpose(0,2,1)
opt_rates = test_data['R_cf_opt_min']

In [8]:
np.mean(opt_rates)

12.72640071899708

In [ ]:
def normalize_data(train_data,test_data):
    train_std = np.std(train_data)
    norm_train = (train_data)/train_std
    norm_test = (test_data)/train_std
    return norm_train, norm_test
norm_train_losses_10_30, norm_test_losses = normalize_data(data_list[0]['beta']**(1/2), beta_test**(1/2) )
norm_train_losses_20_50, _ = normalize_data(data_list[1]['beta']**(1/2), beta_test**(1/2) )
norm_train_losses_20_70, _ = normalize_data(data_list[2]['beta']**(1/2), beta_test**(1/2) )
norm_train_losses_20_100, _ = normalize_data(data_list[3]['beta']**(1/2), beta_test**(1/2) )

In [10]:
test_K, test_M = beta_test.shape[2], beta_test.shape[1]

In [11]:
class PCDataset(Dataset):
    def __init__(self, norm_losses, direct, cross, KM):
        self.norm_losses = norm_losses
        self.direct = torch.tensor(direct, dtype=torch.float)
        self.cross = torch.tensor(cross, dtype=torch.float)
        self.KM = KM
        self.get_cg()
        self.process()

    def build_graph(self, idx):
        edge_feature = self.norm_losses[idx, :, :].reshape((self.KM[0] * self.KM[1], 1), order='F')
        edge_feature = np.concatenate((edge_feature, np.ones_like(edge_feature)), axis=-1)
        edge_feature = torch.tensor(edge_feature, dtype=torch.float)

        edge_index = torch.tensor(self.adj, dtype=torch.long).t().contiguous()
        edge_index_t = torch.tensor(self.adj_t, dtype=torch.long).t().contiguous()
        ue_features = torch.ones((self.KM[0], 1))
        ap_features = torch.ones((self.KM[1], 1))

        data = HeteroData()
        data['UE'].x = ue_features
        data['AP'].x = ap_features
        data['UE', 'com-by', 'AP'].edge_index = edge_index
        data['UE', 'com-by', 'AP'].edge_attr = edge_feature
        data['AP', 'com', 'UE'].edge_index = edge_index_t
        data['AP', 'com', 'UE'].edge_attr = edge_feature

        return data

    def get_cg(self):
        self.adj = []
        self.adj_t = []
        for i in range(self.KM[0]):
            for j in range(self.KM[1]):
                self.adj.append([i, j])
                self.adj_t.append([j, i])

    def process(self):
        self.graph_list = [self.build_graph(i) for i in range(len(self.direct))]

    def __len__(self):
        return len(self.direct)

    def __getitem__(self, index):
        return self.graph_list[index], self.direct[index], self.cross[index]

In [12]:
def collate(samples):
    '''Pytorch Geometric collate function'''
    graphs, direct, cross = map(list, zip(*samples))
    batched_graph = Batch.from_data_list(graphs)
    return batched_graph, torch.stack(direct), torch.stack(cross)

In [13]:
test_data = PCDataset(norm_test_losses, direct_test, cross_test, (test_K, test_M))

In [ ]:
train_data_10_30 = PCDataset(norm_train_losses_10_30, data_list[0]['direct'], data_list[0]['cross'], (10, 30))
train_data_20_50 = PCDataset(norm_train_losses_20_50, data_list[1]['direct'], data_list[1]['cross'], (20, 50))
train_data_10_70 = PCDataset(norm_train_losses_20_70, data_list[2]['direct'], data_list[2]['cross'], (20, 70))
train_data_20_100 = PCDataset(norm_train_losses_20_100, data_list[3]['direct'], data_list[3]['cross'], (20, 100))

In [ ]:
batch_size = 64
test_loader = DataLoader(test_data, test_layouts, shuffle=False, collate_fn=collate)
train_loader_10_30 = DataLoader(train_data_10_30, batch_size, shuffle=True, collate_fn=collate)
train_loader_20_50 = DataLoader(train_data_20_50, batch_size, shuffle=True, collate_fn=collate)
train_loader_20_70 = DataLoader(train_data_10_70, batch_size, shuffle=True, collate_fn=collate)
train_loader_20_100 = DataLoader(train_data_20_100, batch_size, shuffle=True, collate_fn=collate)

In [16]:
def sum_rate_loss(allocs, directlink_channel_losses, crosslink_channel_losses, test_mode = False):
    SINRs_numerators = allocs * directlink_channel_losses**2
    SINRs_denominators = torch.squeeze(torch.matmul(crosslink_channel_losses, torch.unsqueeze(allocs, axis=-1))) + directlink_channel_losses
    SINRs = SINRs_numerators / SINRs_denominators
    rates = torch.log2(1 + SINRs)
    min_rate = torch.sum(rates, dim = 1) # take sum
    if test_mode:
        return min_rate
    else:
        return -torch.mean(min_rate)

In [17]:
def MLP(channels, batch_norm=True):
    return Seq(*[
        Seq(Lin(channels[i - 1], channels[i]), ReLU(), BN(channels[i]))
        for i in range(1, len(channels))
    ])
class EdgeConv(MessagePassing):
    def __init__(self, input_dim, node_dim, **kwargs):
        super(EdgeConv, self).__init__(aggr='mean')  # mean aggregation
        self.lin = MLP([input_dim, 32])
        self.res_lin = Lin(node_dim, 32)
        self.bn = BN(32)

    def forward(self, x, edge_index, edge_attr):

        feat_src, feat_dst = x


        out = self.propagate(edge_index=edge_index, x=(feat_src, feat_dst), edge_attr=edge_attr)


        return self.bn(out + self.res_lin(feat_dst))

    def message(self, x_j, x_i, edge_attr):
        out = torch.cat([x_j, x_i, edge_attr], dim=1)
        return self.lin(out)

    def update(self, aggr_out):
        return aggr_out



In [18]:
class RGCN(nn.Module):
    def __init__(self):
        super(RGCN, self).__init__()
        self.conv1 = HeteroConv({
            ('UE', 'com-by', 'AP'): EdgeConv(4, 1),
            ('AP', 'com', 'UE'): EdgeConv(4, 1)
        }, aggr='sum')

        self.conv2 = HeteroConv({
            ('UE', 'com-by', 'AP'): EdgeConv(66, 32),
            ('AP', 'com', 'UE'): EdgeConv(66, 32)
        }, aggr='sum')

        self.conv3 = HeteroConv({
            ('UE', 'com-by', 'AP'): EdgeConv(66, 32),
            ('AP', 'com', 'UE'): EdgeConv(66, 32)
        }, aggr='sum')

        self.mlp = MLP([32, 16])
        self.mlp = nn.Sequential(*[self.mlp, Seq(Lin(16, 1), Sigmoid())])

    def forward(self,x_dict, edge_index_dict, edge_attr_dict):
        out = self.conv1(x_dict, edge_index_dict, edge_attr_dict)
        out = self.conv2(out, edge_index_dict, edge_attr_dict)
        out = self.conv3(out, edge_index_dict, edge_attr_dict)
        out = self.mlp(out['UE'])
        return out


In [19]:
model = RGCN()

optimizer = torch.optim.Adam(model.parameters(), lr=6e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [ ]:
def train_model(epoch, model, optimizer, train_loader):
    """ Train for one epoch. """
    model.train()
    loss_all = 0
    for batch_idx, (data, d_train, c_train) in enumerate(train_loader):
        K = d_train.shape[-1]
        n = len(data['UE'].x)
        bs = len(data['UE'].x) // K

        optimizer.zero_grad()

        # Lấy các đặc trưng nút từ từ điển x_dict
        user_feats = data['AP'].x
        item_feats = data['UE'].x
        node_features = {'AP': user_feats, 'UE': item_feats}

        # Truyền qua mô hình, bao gồm cả edge_attr_dict
        output = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)

        # Tính loss và thực hiện backpropagation
        loss = sum_rate_loss(output, d_train, c_train)
        loss.backward()

        loss_all += loss.item() * bs
        optimizer.step()

    return loss_all / len(train_loader.dataset)

def test_model(loader, model):
    model.eval()
    correct = 0
    with torch.no_grad():
        for (data, d_test, c_test) in loader:
            K = d_test.shape[-1]
            n = len(data['UE'].x)
            bs = len(data['UE'].x) // K

            # Lấy các đặc trưng nút từ từ điển x_dict
            user_feats = data['AP'].x
            item_feats = data['UE'].x
            # Create a dictionary for node features
            node_features = {'AP': user_feats, 'UE': item_feats}

            output = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)

            # Tính loss
            loss = sum_rate_loss(output, d_test, c_test)
            correct += loss.item() * bs

    return correct / len(loader.dataset)

In [ ]:
record = []
for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            train_rate = test_model(train_loader_10_30, model)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}')
        record.append(train_rate)

    train_model(epoch, model, optimizer, train_loader_10_30)
    scheduler.step()


Epoch 000, Train Rate: -10.4253, Test Rate: -10.3175
Epoch 001, Train Rate: -11.4873, Test Rate: -11.4836
Epoch 002, Train Rate: -11.7386, Test Rate: -11.7523
Epoch 003, Train Rate: -12.0122, Test Rate: -12.0332
Epoch 004, Train Rate: -12.1344, Test Rate: -12.1650
Epoch 005, Train Rate: -12.1755, Test Rate: -12.2090
Epoch 006, Train Rate: -12.2300, Test Rate: -12.2534
Epoch 007, Train Rate: -12.2956, Test Rate: -12.3283
Epoch 008, Train Rate: -12.3248, Test Rate: -12.3553
Epoch 009, Train Rate: -12.1493, Test Rate: -12.1902


In [ ]:
record = []
for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            train_rate = test_model(train_loader_20_50, model)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}')
        record.append(train_rate)

    train_model(epoch, model, optimizer, train_loader_20_50)
    scheduler.step()


Epoch 000, Train Rate: -24.3976, Test Rate: -12.3897
Epoch 001, Train Rate: -23.9537, Test Rate: -12.0969
Epoch 002, Train Rate: -24.4831, Test Rate: -12.3342
Epoch 003, Train Rate: -24.4686, Test Rate: -12.3855
Epoch 004, Train Rate: -24.5785, Test Rate: -12.4484
Epoch 005, Train Rate: -24.2212, Test Rate: -12.3147
Epoch 006, Train Rate: -24.1711, Test Rate: -11.9932
Epoch 007, Train Rate: -23.8728, Test Rate: -11.7351
Epoch 008, Train Rate: -24.5917, Test Rate: -12.4395
Epoch 009, Train Rate: -24.4903, Test Rate: -12.3765


In [ ]:
record = []
for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            train_rate = test_model(train_loader_20_70, model)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}')
        record.append(train_rate)

    train_model(epoch, model, optimizer, train_loader_20_70)
    scheduler.step()


Epoch 000, Train Rate: -18.2169, Test Rate: -12.1619
Epoch 001, Train Rate: -18.3632, Test Rate: -12.3500
Epoch 002, Train Rate: -18.3354, Test Rate: -12.3875
Epoch 003, Train Rate: -18.4178, Test Rate: -12.3399
Epoch 004, Train Rate: -18.4589, Test Rate: -12.4854
Epoch 005, Train Rate: -18.4552, Test Rate: -12.4797
Epoch 006, Train Rate: -18.3583, Test Rate: -12.3905
Epoch 007, Train Rate: -18.4628, Test Rate: -12.4163
Epoch 008, Train Rate: -18.2975, Test Rate: -12.0955
Epoch 009, Train Rate: -18.1952, Test Rate: -11.8459


In [ ]:
record = []
for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            train_rate = test_model(train_loader_20_100, model)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}')
        record.append(train_rate)

    train_model(epoch, model, optimizer, train_loader_20_100)
    scheduler.step()


Epoch 000, Train Rate: -35.1976, Test Rate: -12.3463
Epoch 001, Train Rate: -35.1849, Test Rate: -12.4241
Epoch 002, Train Rate: -35.2093, Test Rate: -12.4763
Epoch 003, Train Rate: -35.0856, Test Rate: -12.3664
Epoch 004, Train Rate: -35.1528, Test Rate: -12.3399
Epoch 005, Train Rate: -35.2645, Test Rate: -12.3851
Epoch 006, Train Rate: -35.2820, Test Rate: -12.3080
Epoch 007, Train Rate: -35.2266, Test Rate: -12.2584
Epoch 008, Train Rate: -34.9962, Test Rate: -12.1790
Epoch 009, Train Rate: -35.3075, Test Rate: -12.4227


#HQGNN

In [32]:
pip install pennylane

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\ADMIN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [37]:
import pennylane as qml
from pennylane import numpy as np

n_qubits = 5
dev = qml.device('default.qubit', wires=n_qubits)


In [ ]:
n_layers_circuit_X = 2
def circuit_X_entangling(weights, n_qubits):
    qml.CRX(weights[-1], wires=[n_qubits-1, 0])
    for i in range(n_qubits-1):
        qml.CRX(weights[i], wires=[i, (i+1)])

@qml.qnode(dev, interface='torch')
def circuit_X(inputs, layer_weights):
    qml.AmplitudeEmbedding(features=inputs, wires=range(n_qubits), normalize=True, pad_with=0.)
    for l in range(n_layers_circuit_X):
        circuit_X_entangling(layer_weights[l], n_qubits)
    return qml.probs(wires=range(n_qubits))
weight_shapes_circuit_X = { "layer_weights": (n_layers_circuit_X, n_qubits)}

In [ ]:
n_layers_circuit_Z = 2
def circuit_Z_entangling(weights, n_qubits):
    qml.CRZ(weights[-1], wires=[n_qubits - 1, 0])
    for i in range(n_qubits - 1):
        qml.CRZ(weights[i], wires=[i, i + 1])

@qml.qnode(dev, interface='torch')
def circuit_Z(inputs, layer_weights):
    qml.AmplitudeEmbedding(features=inputs, wires=range(n_qubits), normalize=True, pad_with=0.)
    for l in range(n_layers_circuit_Z):
        circuit_Z_entangling(layer_weights[l], n_qubits)
    return qml.probs(wires=range(n_qubits))
weight_shapes_circuit_Z = { "layer_weights": (n_layers_circuit_Z, n_qubits)}

In [ ]:
def MLP(channels, batch_norm=True):
    return Seq(*[
        Seq(Lin(channels[i - 1], channels[i]), ReLU(), BN(channels[i]))
        for i in range(1, len(channels))
    ])
class Q_layer(MessagePassing):
    def __init__(self,src_dim, dst_dim, edge_dim, **kwargs):
        super(Q_layer, self).__init__(aggr='mean')  # mean aggregation
        self.lin_res = qml.qnn.TorchLayer(circuit_Z, weight_shapes_circuit_Z)
        self.lin_qml = qml.qnn.TorchLayer(circuit_X, weight_shapes_circuit_X)
        self.in_linear = nn.Linear(src_dim + dst_dim + edge_dim, 2 ** n_qubits)
        self.bn = BN(2 ** n_qubits)

    def forward(self, x, edge_index, edge_attr):
        feat_src, feat_dst = x
        out = self.propagate(edge_index=edge_index, x=(feat_src, feat_dst), edge_attr=edge_attr)
        out = out + self.lin_res(feat_dst)
        return self.bn(out)

    def message(self, x_j, x_i, edge_attr):
        out = torch.cat([x_j, x_i, edge_attr], dim=1)
        out = self.in_linear(out)
        out = self.lin_qml(out)
        return out

    def update(self, aggr_out):
        return aggr_out

In [ ]:
class RGCN_Hybrid_mid(nn.Module):
    def __init__(self):
        super(RGCN_Hybrid_mid, self).__init__()
        out_dim = 2**n_qubits
        edge_dim = 2

        self.conv1 = HeteroConv({
            ('UE', 'com-by', 'AP'): Q_layer(1, 1, edge_dim),
            ('AP', 'com', 'UE'): Q_layer(1, 1, edge_dim,)
        }, aggr='mean')

        self.conv2 = HeteroConv({
            ('UE', 'com-by', 'AP'): Q_layer(out_dim, out_dim, edge_dim),
            ('AP', 'com', 'UE'): Q_layer(out_dim, out_dim, edge_dim)
        }, aggr='mean')

        self.conv3 = HeteroConv({
             ('UE', 'com-by', 'AP'): Q_layer(out_dim, out_dim, edge_dim),
             ('AP', 'com', 'UE'): Q_layer(out_dim, out_dim, edge_dim)
         }, aggr='mean')


        self.mlp = MLP([32, 16])
        self.mlp = nn.Sequential(*[self.mlp, Seq(Lin(16, 1), Sigmoid())])
    def forward(self, x_dict, edge_index_dict, edge_attr_dict):
        out = self.conv1(x_dict, edge_index_dict, edge_attr_dict)
        out = self.conv2(out, edge_index_dict, edge_attr_dict)
        out = self.conv3(out, edge_index_dict, edge_attr_dict)
        out = self.mlp(out['UE'])
        return out

In [42]:
model_qml_mid = RGCN_Hybrid_mid().to()

optimizer_qml_mid = torch.optim.Adam(model_qml_mid.parameters(), lr=7e-5)
scheduler_qml_mid = torch.optim.lr_scheduler.StepLR(optimizer_qml_mid, step_size=10, gamma=0.9)

In [ ]:
record = []
for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            train_rate = test_model(train_loader_10_30, model_qml_mid)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}')
        record.append(train_rate)

    train_model(epoch, model_qml_mid, optimizer_qml_mid, train_loader_10_30)
    scheduler_qml_mid.step()

Epoch 000, Train Rate: -10.4198, Test Rate: -10.3122
Epoch 001, Train Rate: -10.4695, Test Rate: -10.3659
Epoch 002, Train Rate: -11.2613, Test Rate: -11.2071
Epoch 003, Train Rate: -11.4383, Test Rate: -11.4095
Epoch 004, Train Rate: -11.5577, Test Rate: -11.5621
Epoch 005, Train Rate: -11.6780, Test Rate: -11.7024
Epoch 006, Train Rate: -11.7422, Test Rate: -11.7803
Epoch 007, Train Rate: -11.8043, Test Rate: -11.8675
Epoch 008, Train Rate: -11.8651, Test Rate: -11.9458
Epoch 009, Train Rate: -11.9313, Test Rate: -12.0201


In [ ]:
record = []
for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            train_rate = test_model(train_loader_20_50, model_qml_mid)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}')
        record.append(train_rate)

    train_model(epoch, model_qml_mid, optimizer_qml_mid, train_loader_20_50)
    scheduler_qml_mid.step()

Epoch 000, Train Rate: -23.3666, Test Rate: -12.0699
Epoch 001, Train Rate: -23.5027, Test Rate: -12.0902
Epoch 002, Train Rate: -23.5933, Test Rate: -12.1281
Epoch 003, Train Rate: -23.7018, Test Rate: -12.1522
Epoch 004, Train Rate: -23.7889, Test Rate: -12.1721
Epoch 005, Train Rate: -23.9000, Test Rate: -12.1998
Epoch 006, Train Rate: -23.9118, Test Rate: -12.1624
Epoch 007, Train Rate: -24.0280, Test Rate: -12.2081
Epoch 008, Train Rate: -23.9205, Test Rate: -12.1639
Epoch 009, Train Rate: -24.2217, Test Rate: -12.3012


In [ ]:
record = []
for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            train_rate = test_model(train_loader_20_70, model_qml_mid)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}')
        record.append(train_rate)

    train_model(epoch, model_qml_mid, optimizer_qml_mid, train_loader_20_70)
    scheduler_qml_mid.step()

Epoch 000, Train Rate: -18.0358, Test Rate: -12.2326
Epoch 001, Train Rate: -18.1848, Test Rate: -12.2705
Epoch 002, Train Rate: -18.1878, Test Rate: -12.2813
Epoch 003, Train Rate: -18.2081, Test Rate: -12.2944
Epoch 004, Train Rate: -18.1681, Test Rate: -12.2721
Epoch 005, Train Rate: -18.1805, Test Rate: -12.2565
Epoch 006, Train Rate: -18.1886, Test Rate: -12.2689
Epoch 007, Train Rate: -18.2103, Test Rate: -12.2838
Epoch 008, Train Rate: -18.2562, Test Rate: -12.3333
Epoch 009, Train Rate: -18.2518, Test Rate: -12.3181


In [ ]:
record = []
for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            train_rate = test_model(train_loader_20_100, model_qml_mid)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}')
        record.append(train_rate)

    train_model(epoch, model_qml_mid, optimizer_qml_mid, train_loader_20_100)
    scheduler_qml_mid.step()

Epoch 000, Train Rate: -34.6574, Test Rate: -12.3109
Epoch 001, Train Rate: -34.2959, Test Rate: -12.1161
Epoch 002, Train Rate: -34.4878, Test Rate: -12.1919
Epoch 003, Train Rate: -34.6850, Test Rate: -12.2000
Epoch 004, Train Rate: -34.9084, Test Rate: -12.2639
Epoch 005, Train Rate: -34.8823, Test Rate: -12.2671
Epoch 006, Train Rate: -34.9449, Test Rate: -12.2849
Epoch 007, Train Rate: -35.0062, Test Rate: -12.2787
Epoch 008, Train Rate: -34.9877, Test Rate: -12.2870
Epoch 009, Train Rate: -34.4705, Test Rate: -12.1174


In [ ]:
torch.save(model.state_dict(), 'model_sum.pth')
torch.save(model_qml_mid.state_dict(), 'model_amplitude_sum.pth')

In [ ]:
opt_20 = []
qgnn_20 = []
gnn_20 = []
all_one_rates_20 = []
for i in range(30,101,10):
  open_file = str(i) + '_20.mat'
  data = scipy.io.loadmat(open_file)
  beta = data['betas']
  direct = data['directs']
  cross = data['corsses'].transpose(0,2,1)
  opti = data['R_cf_opt_min']
  opt_20.append(np.mean(opti))
  _, norm_losses = normalize_data(beta**(1/2), beta**(1/2))
  data = PCDataset(norm_losses, direct, cross, (20, i))
  loader = DataLoader(data, 100, shuffle=False, collate_fn=collate)
  for (data, d_test, c_test) in loader:
    K = d_test.shape[-1]
    n = len(data['UE'].x)
    bs = len(data['UE'].x) // K

    user_feats = data['AP'].x
    item_feats = data['UE'].x
    node_features = {'AP': user_feats, 'UE': item_feats}

    output = model_qml_mid(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)
    output1 = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)
    full = torch.ones_like(output)
    qgnn_rates = -sum_rate_loss(output, d_test, c_test).detach().numpy()
    gnn_rates = -sum_rate_loss(output1, d_test, c_test).detach().numpy()
    all_one = -sum_rate_loss(full, d_test, c_test).detach().numpy()
  gnn_20.append(gnn_rates)
  qgnn_20.append(qgnn_rates)
  all_one_rates_20.append(all_one)

import matplotlib.pyplot as plt


plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams.update({'font.size': 12})


fig, ax = plt.subplots(figsize=(6, 6), dpi=180)
x_values = list(range(30, 101, 10))

ax.plot(x_values, qgnn_20, label='HQGNN', linestyle=':', marker='o')  
ax.plot(x_values, gnn_20, label='GNN', linestyle='--')
ax.plot(x_values, opt_20, label='Optimization', linestyle='-') 
ax.plot(x_values, all_one_rates_20, label='Maximum Power', linestyle='-', marker='^')


ax.set_xlabel('Number APs')
ax.set_ylabel('Sum-rate')


ax.legend()
ax.grid()


x1, x2, y1, y2 = 79, 81, 30.7, 31.7


axins = ax.inset_axes([0.75, 0.3, 0.2, 0.2])


axins.plot(x_values, qgnn_20, label='HQGNN', linestyle=':', marker='o')
axins.plot(x_values, gnn_20, label='GNN', linestyle='--')
axins.plot(x_values, opt_20, label='Optimization', linestyle='-')
axins.plot(x_values, all_one_rates_20, label='Maximum Power', linestyle='-', marker='^')


axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)


ax.indicate_inset_zoom(axins, edgecolor="black")

plt.show()


# HQGNN Angle

In [ ]:
n_layers_circuit_X2 = 4
def circuit_X2_entangling(weights, n_qubits):
    qml.CRX(weights[-1], wires=[n_qubits-1, 0])
    for i in range(n_qubits-1):
        qml.CRX(weights[i], wires=[i, (i+1)])
@qml.qnode(dev, interface='torch')
def circuit_X2(inputs, layer_weights):
    qml.AngleEmbedding(features=inputs, wires=range(n_qubits))
    for l in range(n_layers_circuit_X2):
        circuit_X2_entangling(layer_weights[l], n_qubits)
    return qml.probs(wires=range(n_qubits))
weight_shapes_circuit_X2 = { "layer_weights": (n_layers_circuit_X2, n_qubits)}

In [ ]:
n_layers_circuit_Z2 = 4
def circuit_Z2_entangling(weights, n_qubits):
    qml.CRZ(weights[-1], wires=[n_qubits - 1, 0])
    for i in range(n_qubits - 1):
        qml.CRZ(weights[i], wires=[i, i + 1])

@qml.qnode(dev, interface='torch')
def circuit_Z2(inputs, layer_weights):
    qml.AngleEmbedding(features=inputs, wires=range(n_qubits))
    for l in range(n_layers_circuit_Z2):
        circuit_Z2_entangling(layer_weights[l], n_qubits)
    return qml.probs(wires=range(n_qubits))
weight_shapes_circuit_Z2 = { "layer_weights": (n_layers_circuit_Z2, n_qubits)}

In [ ]:
def MLP(channels, batch_norm=True):
    return Seq(*[
        Seq(Lin(channels[i - 1], channels[i]), ReLU(), BN(channels[i]))
        for i in range(1, len(channels))
    ])
class Q_layer_angle(MessagePassing):
    def __init__(self,src_dim, dst_dim, edge_dim, out_dim, **kwargs):
        super(Q_layer_angle, self).__init__(aggr='mean')  
        self.lin_res = qml.qnn.TorchLayer(circuit_X2, weight_shapes_circuit_X2)
        self.lin_qml = qml.qnn.TorchLayer(circuit_Z2, weight_shapes_circuit_Z2)
        self.in_linear = nn.Linear(src_dim + dst_dim + edge_dim, out_dim)
        self.linear = nn.Linear(dst_dim, n_qubits)
        self.bn = BN(32)

    def forward(self, x, edge_index, edge_attr):
        feat_src, feat_dst = x
        out = self.propagate(edge_index=edge_index, x=(feat_src, feat_dst), edge_attr=edge_attr)
        out = out + self.lin_res(self.linear(feat_dst))
        return self.bn(out)

    def message(self, x_j, x_i, edge_attr):
        out = torch.cat([x_j, x_i, edge_attr], dim=1)
        out = self.in_linear(out)
        out = self.lin_qml(out)
        return out

    def update(self, aggr_out):
        return aggr_out

In [ ]:
class RGCN_Hybrid_mid_angle(nn.Module):
    def __init__(self):
        super(RGCN_Hybrid_mid_angle, self).__init__()
        out_dim = n_qubits
        edge_dim = 2

        self.conv1 = HeteroConv({
            ('UE', 'com-by', 'AP'): Q_layer_angle(1, 1, edge_dim, out_dim),
            ('AP', 'com', 'UE'): Q_layer_angle(1, 1, edge_dim,out_dim)
        }, aggr='mean')

        self.conv2 = HeteroConv({
            ('UE', 'com-by', 'AP'): Q_layer_angle(32, 32, edge_dim, out_dim),
            ('AP', 'com', 'UE'): Q_layer_angle(32, 32, edge_dim, out_dim)
        }, aggr='mean')

        self.conv3 = HeteroConv({
             ('UE', 'com-by', 'AP'): Q_layer_angle(32, 32, edge_dim, out_dim),
             ('AP', 'com', 'UE'): Q_layer_angle(32, 32, edge_dim, out_dim)
         }, aggr='mean')

        self.mlp = MLP([32, 16])
        self.mlp = nn.Sequential(*[self.mlp, Seq(Lin(16, 1), Sigmoid())])
    def forward(self, x_dict, edge_index_dict, edge_attr_dict):
        out = self.conv1(x_dict, edge_index_dict, edge_attr_dict)
        out = self.conv2(out, edge_index_dict, edge_attr_dict)
        out = self.conv3(out, edge_index_dict, edge_attr_dict)
        out = self.mlp(out['UE'])
        return out

In [ ]:
model_qml_mid_angle = RGCN_Hybrid_mid_angle().to()

optimizer_qml_mid_angle = torch.optim.Adam(model_qml_mid_angle.parameters(), lr=5e-4)
scheduler_qml_mid_angle = torch.optim.lr_scheduler.StepLR(optimizer_qml_mid_angle, step_size=10, gamma=0.4)
trainable_params = sum(p.numel() for p in model_qml_mid_angle.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")

In [ ]:
record = []
for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            train_rate = test_model(train_loader_20_50, model_qml_mid_angle)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}')
        record.append(train_rate)

    train_model(epoch, model_qml_mid_angle, optimizer_qml_mid_angle, train_loader_20_50)
    scheduler_qml_mid_angle.step()

In [ ]:
record = []
for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            train_rate = test_model(train_loader_20_70, model_qml_mid_angle)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}')
        record.append(train_rate)

    train_model(epoch, model_qml_mid_angle, optimizer_qml_mid_angle, train_loader_20_70)
    scheduler_qml_mid_angle.step()

In [ ]:
record = []
for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            train_rate = test_model(train_loader_20_100, model_qml_mid_angle)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}')
        record.append(train_rate)

    train_model(epoch, model_qml_mid_angle, optimizer_qml_mid_angle, train_loader_20_100)
    scheduler_qml_mid_angle.step()

In [ ]:
torch.save(model_qml_mid_angle.state_dict(), 'model_angle_sum.pth')

In [ ]:
# model.load_state_dict(torch.load('model_sum.pth'))
# model_qml_mid.load_state_dict(torch.load('model_amp_sum.pth'))

In [ ]:
angle_qgnn_20 = []
opt_20 = []
all_one_rates_20 = []
random_rates = []
for i in range(30,101,10):
  open_file = str(i) + '_20.mat'
  data = scipy.io.loadmat(open_file)
  beta = data['betas']
  direct = data['directs']
  cross = data['corsses'].transpose(0,2,1)
  opti = data['R_cf_opt_min']
  opt_20.append(np.mean(opti))
  _, norm_losses = normalize_data(beta**(1/2), beta**(1/2))
  data = PCDataset(norm_losses, direct, cross, (20, i))
  loader = DataLoader(data, 100, shuffle=False, collate_fn=collate)
  for (data, d_test, c_test) in loader:
    K = d_test.shape[-1]
    n = len(data['UE'].x)
    bs = len(data['UE'].x) // K

    user_feats = data['AP'].x
    item_feats = data['UE'].x
    node_features = {'AP': user_feats, 'UE': item_feats}

    output = model_qml_mid_angle(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)
    full = torch.ones_like(output)
    random = torch.rand_like(output)
    qgnn_rates = -sum_rate_loss(output, d_test, c_test).detach().numpy()
    all_one = -sum_rate_loss(full, d_test, c_test).detach().numpy()
    rand_rates = -sum_rate_loss(random, d_test, c_test).detach().numpy()
    
  
  angle_qgnn_20.append(qgnn_rates)
  all_one_rates_20.append(all_one)
  random_rates.append(rand_rates)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


def count_parameters(model):
    quantum_params = 0
    classical_params = 0
    for name, param in model.named_parameters():
        if 'lin_qml' in name or 'lin_res' in name:  
            quantum_params += param.numel()
        else:  
            classical_params += param.numel()
    return quantum_params, classical_params

models = {
    "GNN": model,
    "HQGNN Amplitude": model_qml_mid,
    "HQGNN Angle": model_qml_mid_angle,
}

params_count = {
    name: count_parameters(model)
    for name, model in models.items()
}



avg_gnn_test_rates_amplitude = np.mean(qgnn_20)
avg_gnn_test_rates = np.mean(gnn_20)
opt_rates = np.mean(opt_20)
avg_gnn_test_rates_angle = np.mean(angle_qgnn_20)
    
performance_percentages = {
    "GNN": (avg_gnn_test_rates / opt_rates) * 100,
    "HQGNN Amplitude": (avg_gnn_test_rates_amplitude / opt_rates) * 100,
    "HQGNN Angle": (avg_gnn_test_rates_angle / opt_rates)*100,
}


data = {
    "Model": ["GNN", "HQGNN Amplitude", "HQGNN Angle"],
    "Classical Parameters": [params_count["GNN"][1], params_count["HQGNN Amplitude"][1], params_count["HQGNN Angle"][1]],
    "Quantum Parameters": [params_count["GNN"][0], params_count["HQGNN Amplitude"][0],params_count["HQGNN Angle"][0]],
    "Total Parameters": [params_count["GNN"][0] + params_count["GNN"][1],params_count["HQGNN Amplitude"][0]+params_count["HQGNN Amplitude"][1], params_count["HQGNN Angle"][1] + params_count["HQGNN Angle"][0]],
    "Performance (%)": [performance_percentages["GNN"], performance_percentages["HQGNN Amplitude"], performance_percentages["HQGNN Angle"]],
}
params_df = pd.DataFrame(data)


fig, ax = plt.subplots(figsize=(8, 4))  
ax.axis('tight')  
ax.axis('off')  
table = ax.table(cellText=params_df.values, colLabels=params_df.columns, loc='center', cellLoc='center')


table.auto_set_font_size(False)
table.set_fontsize(12)
table.auto_set_column_width(col=list(range(len(params_df.columns))))

for (row, col), cell in table.get_celld().items():
    cell.set_linewidth(1.5)  # Độ dày đường kẻ
    cell.set_height(0.3)    # Tăng chiều cao hàng
    cell.set_width(0.25) 

# Hiển thị bảng
plt.savefig("model_parameters_table_from_code.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def count_parameters(model):
    quantum_params = 0
    classical_params = 0
    for name, param in model.named_parameters():
        if 'lin_qml' in name or 'lin_res' in name:  
            quantum_params += param.numel()
        else:  
            classical_params += param.numel()
    return quantum_params, classical_params

# Models to analyze
models = {
    "GNN": model,
    "HQGNN Amplitude": model_qml_mid,
    "HQGNN Angle": model_qml_mid_angle,
}

# Compute parameter counts
params_count = {
    name: count_parameters(model)
    for name, model in models.items()
}

# Performance metrics
avg_gnn_test_rates = np.mean(gnn_20)
avg_gnn_test_rates_amplitude = np.mean(qgnn_20)
avg_gnn_test_rates_angle = np.mean(angle_qgnn_20)
opt_rates = np.mean(opt_20)
all_one_rates = np.mean(all_one_rates_20)
rand_rates = np.mean(random_rates)

performance_percentages = {
    "GNN": (avg_gnn_test_rates / opt_rates) * 100,
    "HQGNN Amplitude": (avg_gnn_test_rates_amplitude / opt_rates) * 100,
    "HQGNN Angle": (avg_gnn_test_rates_angle / opt_rates) * 100,
    "Maximum Power": (all_one_rates / opt_rates) * 100,
    "Random Power": (rand_rates / opt_rates) * 100
}

# Combine data into a DataFrame
data = {
    "Model": ["GNN", "HQGNN Amplitude", "HQGNN Angle", "Maximum Power", "Random Power"],
    "Classical Parameters": [
        params_count["GNN"][1], 
        params_count["HQGNN Amplitude"][1], 
        params_count["HQGNN Angle"][1], 
        0, 
        0
    ],
    "Quantum Parameters": [
        params_count["GNN"][0], 
        params_count["HQGNN Amplitude"][0], 
        params_count["HQGNN Angle"][0], 
        0, 
        0
    ],
    "Total Parameters": [
        params_count["GNN"][0] + params_count["GNN"][1], 
        params_count["HQGNN Amplitude"][0] + params_count["HQGNN Amplitude"][1], 
        params_count["HQGNN Angle"][0] + params_count["HQGNN Angle"][1], 
        0, 
        0
    ],
    "Performance (%)": [
        performance_percentages["GNN"], 
        performance_percentages["HQGNN Amplitude"], 
        performance_percentages["HQGNN Angle"], 
        performance_percentages["Maximum Power"], 
        performance_percentages["Random Power"]
    ],
}

params_df = pd.DataFrame(data)

# Visualize the data as a table
fig, ax = plt.subplots(figsize=(10, 5))
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=params_df.values, colLabels=params_df.columns, loc='center', cellLoc='center')

table.auto_set_font_size(False)
table.set_fontsize(12)
table.auto_set_column_width(col=list(range(len(params_df.columns))))

for (row, col), cell in table.get_celld().items():
    cell.set_linewidth(1.5)
    cell.set_height(0.3)
    cell.set_width(0.25)

# Save and display the table
plt.savefig("model_parameters_table_combined.png", dpi=300, bbox_inches='tight')
plt.show()

import matplotlib.pyplot as plt


plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams.update({'font.size': 12})


fig, ax = plt.subplots(figsize=(6, 6), dpi=180)
x_values = list(range(30, 101, 10))

ax.plot(x_values, qgnn_20, label='HQGNN Amplitude', color='blue', linestyle=':', marker='o', markersize=5)  
ax.plot(x_values, gnn_20, label='GNN', color='orange', linestyle='--', markersize=5)
ax.plot(x_values, angle_qgnn_20, label='HQGNN Angle', color='green', linestyle='-', marker='x', markersize=5)
ax.plot(x_values, opt_20, label='Optimization', color='red', linestyle='-')
ax.plot(x_values, all_one_rates_20, label='Maximum Power', color='purple', marker='^', markersize=5)


ax.set_xlabel('Number APs')
ax.set_ylabel('Sum-rate')


ax.legend()
ax.grid()


x1, x2, y1, y2 = 69, 71, 29, 30


axins = ax.inset_axes([0.75, 0.3, 0.2, 0.2])


axins.plot(x_values, qgnn_20, label='HQGNN Amplitude', linestyle=':', marker='o')
axins.plot(x_values, gnn_20, label='GNN', linestyle='--')
axins.plot(x_values, angle_qgnn_20, label='HQGNN Angle', marker='x')
axins.plot(x_values, opt_20, label='Optimization', linestyle='-')
axins.plot(x_values, all_one_rates_20, label='Maximum Power', linestyle='-', marker='^')


axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)


ax.indicate_inset_zoom(axins, edgecolor="black")

plt.show()